First we have to import the necessary functionality from pycram.

In [1]:
from pycram.bullet_world import BulletWorld
from pycram.robot_descriptions.robot_description_handler import InitializedRobotDescription as robot_description
import pycram.task
from pycram.resolver.plans import Arms
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot

ModuleNotFoundError: No module named 'pybullet'

Next we will create a bullet world with a PR2 in a kitchen containing milk and cereal.

In [ ]:
world = BulletWorld()
robot = Object(robot_description.i.name, "robot", robot_description.i.name + ".urdf")
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=[1.3, 0.7, 0.95])
milk_desig = ObjectDesignator(Object(name="milk", type="milk"))
cereal_desig = ObjectDesignator(Object(name="cereal", type="cereal"))

Finally, we create a plan where the robot parks his arms, walks to the kitchen counter and picks the thingy. Then we execute the plan.

In [ ]:
@pycram.task.with_tree
def plan():
    with simulated_robot:
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

        location = LocationDesignator(CostmapLocation(target=milk, reachable_for=robot))
        pose = location.reference()
        ActionDesignator(
            NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

        picked_up_arm = pose["arms"][0]
        ActionDesignator(PickUpAction(object_designator=milk_desig, arm=pose["arms"][0], grasp="front")).perform()

        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()
        place_island = LocationDesignator(SemanticCostmapLocation("kitchen_island_surface", kitchen, milk_desig.prop_value("object")))
        pose_island = place_island.reference()

        place_location = LocationDesignator(CostmapLocation(target=list(pose_island.values()), reachable_for=robot, reachable_arm=picked_up_arm))
        pose = place_location.reference()

        ActionDesignator(NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()

        ActionDesignator(PlaceAction(object_designator=milk_desig, target_location=list(pose_island.values()), arm=picked_up_arm)).perform()

        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

plan()